In [109]:
with open('15a.txt') as f:
    lines = f.readlines()

lines = [line.strip() for line in lines]

In [110]:
import numpy as np
grid = np.zeros((len(lines),len(lines[0])))
for i in range(len(lines)):
    for j in range(len(lines[i])):
        grid[i][j] = lines[i][j]

### Part 1

In [111]:
# could literally try every path and track the total risk level, then compare. naive method
# there's no situation in going on an already traversed number is good.

In [112]:
# failed attempt to just traverse the whole thing recursively. not part of final solution.
def traverse(grid, row, col, visited):
    new_visited = visited.copy()
    # print(len(visited))
    new_sum = grid[row][col]
    path_vals = []
    if row == len(grid)-1 and col == len(grid[0])-1:
        return new_sum
    to_traverse = []
    for r in range(row-1,row+2):
        if r >= 0 and r < len(grid):
            for c in range(col-1, col+2):
                if c >= 0 and c < len(grid[r]):
                    if (r == row or c == col) and (r != row or c != col) and (not (r,c) in visited):
                        # print("row: " + str(r))
                        # print("col: " + str(c))
                        new_visited.append((r,c))
                        to_traverse.append((r,c))
    print("(" + str(row) + "," + str(col) + ") # of traverse: " + str(len(to_traverse)))
    for coords in to_traverse:
        path_vals.append(new_sum + traverse(grid,coords[0],coords[1],new_visited))
    if len(to_traverse) == 0:
        return 999999
    return min(path_vals)


In [113]:
def find_best_unvisited(unvisited, best_path_to_origin):
    min_risk = best_path_to_origin[unvisited[0]]
    best_unvisited = unvisited[0]
    for coord in unvisited:
        if best_path_to_origin[coord] < min_risk:
            min_risk = best_path_to_origin[coord]
            best_unvisited = coord
    return best_unvisited

In [130]:
## ok we gonna do some dijkstra algo
def least_risk_path(grid):
    unvisited = []
    best_path_to_origin = {}
    path_to_best = {}
    for row in range(len(grid)):
        for col in range(len(grid[row])):
            best_path_to_origin[(row,col)] = float('inf')
            unvisited.append((row,col))
            path_to_best[(row,col)] = (0,0)
    best_path_to_origin[(0,0)] = 0
    curr_coord = (0,0)
    while(len(unvisited) > 1):
        row = curr_coord[0]
        col = curr_coord[1]
        neighbours = []
        for r in range(row-1,row+2):
                if r >= 0 and r < len(grid):
                    for c in range(col-1, col+2):
                        if c >= 0 and c < len(grid[r]):
                            if (r == row or c == col) and (r != row or c != col) and ((r,c) in unvisited):
                                neighbours.append((r,c))
        for nb in neighbours:
            curr_risk = grid[nb] + best_path_to_origin[(row,col)]
            if curr_risk < best_path_to_origin[nb]:
                best_path_to_origin[nb] = curr_risk
                path_to_best[nb] = (row,col)
        unvisited.remove((row,col))
        curr_coord = find_best_unvisited(unvisited, best_path_to_origin)
        # print(1 - len(unvisited)/(500*500.0))
    temp = (len(grid)-1, len(grid)-1)
    while (temp != (0,0)):
        new_temp = path_to_best[temp]
        if new_temp[0] - temp[0] == 1:
            print("went up lol")
        elif new_temp[1] - temp[1] == 1:
            print("went left lol")
        temp = new_temp
    return best_path_to_origin[(len(grid)-1,len(grid[0])-1)]

In [131]:
print(least_risk_path(grid))

went left lol
619.0


### Part 2

In [116]:
def calculate_grid2_val (old_grid_val, i, j):
    num = old_grid_val + i + j
    if num >= 10: num -= 9
    return num

In [117]:
grid2 = np.zeros((len(grid)* 5, len(grid[0])*5))
for i in range(0,5):
    for j in range(0,5):
        for row in range(len(grid)):
            for col in range(len(grid[row])):
                new_row = row + len(grid) * i
                new_col = col + len(grid[row]) * j
                grid2[new_row][new_col] = calculate_grid2_val(grid[row][col], i, j)

In [118]:
## ok we gonna do some dijkstra algo
def least_risk_path(grid):
    unvisited = []
    best_path_to_origin = {}
    path_to_best = {}
    length_path_to_best = {}
    for row in range(len(grid)):
        for col in range(len(grid[row])):
            best_path_to_origin[(row,col)] = float('inf')
            unvisited.append((row,col))
            path_to_best[(row,col)] = (0,0)
            length_path_to_best[(row,col)] = 0
    best_path_to_origin[(0,0)] = 0
    curr_coord = (0,0)
    while(len(unvisited) > 1):
        row = curr_coord[0]
        col = curr_coord[1]
        neighbours = []
        for r in range(row-1,row+2):
                if r >= 0 and r < len(grid):
                    for c in range(col-1, col+2):
                        if c >= 0 and c < len(grid[r]):
                            if (r == row or c == col) and (r != row or c != col) and ((r,c) in unvisited):
                                neighbours.append((r,c))
        for nb in neighbours:
            curr_risk = grid[nb] + best_path_to_origin[(row,col)]
            if curr_risk < best_path_to_origin[nb]:
                best_path_to_origin[nb] = curr_risk
                path_to_best[nb] = (row,col)
        unvisited.remove((row,col))
        curr_coord = find_best_unvisited(unvisited, best_path_to_origin)
        print(1 - len(unvisited)/(500*500.0))
        # figure out how to find length of best path, try all entries and exits. shouldnt be that bad??


In [119]:
def marcos_algorithm(grid):
    risk_to_end = np.zeros((len(grid), len(grid[0])))
    for row in range(len(grid)-1, -1, -1):
        for col in range(len(grid[row])-1, -1, -1):
            down = float('inf')
            right = float('inf')
            if row+1 < len(grid):
                down = risk_to_end[row+1][col]
            if col+1 < len(grid[0]):
                right = risk_to_end[row][col+1]
            if (down == float('inf') and right == float('inf')): down = 0
            risk_to_end[row][col] = grid[row][col] + min(down, right)
    # return risk_to_end[0][0] - grid[0][0]
    return risk_to_end

In [142]:
def improved_marcos_algorithm(grid, risk_grid):
    risk_to_end = np.zeros((len(grid), len(grid[0])))
    for row in range(len(grid)-1, -1, -1):
        for col in range(len(grid[row])-1, -1, -1):
            down = float('inf')
            right = float('inf')
            left = float('inf')
            up = float('inf')
            if row+1 < len(grid):
                down = risk_to_end[row+1][col]
            if col+1 < len(grid[0]):
                right = risk_to_end[row][col+1]
            if row-1 > 0:
                up = risk_grid[row-1][col]
            if col-1 > 0:
                left = risk_grid[row][col-1]
            if (down == float('inf') and right == float('inf')): down = 0
            risk_to_end[row][col] = grid[row][col] + min(down, right, left, up)
    # return risk_to_end[0][0] - grid[0][0]
    return risk_to_end

In [152]:
rte = marcos_algorithm(grid2)
rte2 = improved_marcos_algorithm(grid2, rte)
rte3 = improved_marcos_algorithm(grid2, rte2)
rte4 = improved_marcos_algorithm(grid2, rte3)
for i in range(35):
    rte4 = improved_marcos_algorithm(grid2, rte4)
print(rte4)
# 1789 too low.
# 2926 too high.

[[2926. 2924. 2923. ... 2160. 2161. 2167.]
 [2922. 2921. 2914. ... 2159. 2164. 2165.]
 [2922. 2919. 2913. ... 2156. 2157. 2166.]
 ...
 [2163. 2158. 2156. ...   30.   22.   21.]
 [2171. 2167. 2163. ...   21.   15.   13.]
 [2173. 2168. 2164. ...   13.   10.    6.]]
